In [1]:
import tensorflow as tf
import numpy as np

from functools import partial

## tf.get_collection()  

tf.get_collection()  を利用すれば、グラフの変数を取得することができる。

引数には、
* tf.GraphKeys.TRAINABLE_VARIABLES
* tf.GraphKeys.GLOBAL_VARIABLES
などを設定する。  
https://www.tensorflow.org/api_docs/python/tf/GraphKeys



### set parameters

In [2]:
n_input = 200
n_hidden_1 = 100

## Fully 

### define placeholder & variables

In [3]:
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, [None, n_input])

l2_reg = 0.001
he_init = tf.contrib.layers.variance_scaling_initializer() # He initialization
l2_regularizer = tf.contrib.layers.l2_regularizer(l2_reg) 
with tf.variable_scope('layers1'):    
    decorder_1 = tf.layers.dense(X, n_hidden_1, kernel_initializer=he_init, kernel_regularizer=l2_regularizer)
with tf.variable_scope('layers2'):     
    decorder_2 = tf.layers.dense(X, n_hidden_1, kernel_initializer=he_init, kernel_regularizer=l2_regularizer)

### グラフに追加されたVariable情報を取得

In [4]:
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)

<tf.Variable 'layers1/dense/kernel:0' shape=(200, 100) dtype=float32_ref>
<tf.Variable 'layers1/dense/bias:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'layers2/dense/kernel:0' shape=(200, 100) dtype=float32_ref>
<tf.Variable 'layers2/dense/bias:0' shape=(100,) dtype=float32_ref>


### 特定スコープのVariable情報を取得

In [5]:
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'layers1'):
    print(v.name)

layers1/dense/kernel:0
layers1/dense/bias:0


## Convolutional Nerual Network

In [6]:
with tf.variable_scope('input'):
    inputs_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
    
conv1 = tf.layers.conv2d(inputs_, 16, (3,3), padding='same', activation=tf.nn.relu)    

In [7]:
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)

<tf.Variable 'layers1/dense/kernel:0' shape=(200, 100) dtype=float32_ref>
<tf.Variable 'layers1/dense/bias:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'layers2/dense/kernel:0' shape=(200, 100) dtype=float32_ref>
<tf.Variable 'layers2/dense/bias:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 16) dtype=float32_ref>
<tf.Variable 'conv2d/bias:0' shape=(16,) dtype=float32_ref>


## スコープ内の変数を取得する方法1

In [8]:
t_vars = tf.trainable_variables()

In [9]:
t_vars

[<tf.Variable 'layers1/dense/kernel:0' shape=(200, 100) dtype=float32_ref>,
 <tf.Variable 'layers1/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'layers2/dense/kernel:0' shape=(200, 100) dtype=float32_ref>,
 <tf.Variable 'layers2/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(16,) dtype=float32_ref>]

In [10]:
g_vars = [var for var in t_vars if var.name.startswith('layers1')]

In [11]:
g_vars

[<tf.Variable 'layers1/dense/kernel:0' shape=(200, 100) dtype=float32_ref>,
 <tf.Variable 'layers1/dense/bias:0' shape=(100,) dtype=float32_ref>]

## スコープ内の変数を取得する方法2

格納された順番で変数の値がリストとして格納される  
[0] : 'layers/dense/kernel:0'  
[1] : 'layers/dense/bias:0'  
[2] : 'layers/dense_1/kernel:0'  
[3] : 'layers/dense_1/bias:0'  

In [12]:
layer1_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'layers1')

In [15]:
layer1_vars[0]

<tf.Variable 'layers1/dense/kernel:0' shape=(200, 100) dtype=float32_ref>

In [16]:
layer1_vars[1]

<tf.Variable 'layers1/dense/bias:0' shape=(100,) dtype=float32_ref>

In [17]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    W1 = sess.run([layer1_vars[0]])

(※）なぜ、W1の値にnanが含まれるのかは不明。

In [18]:
print(W1)

[array([[ 0.05459764,  0.08292954, -0.07612792, ...,  0.02462619,
         0.18442191,  0.05535779],
       [-0.10509244,  0.22576694,  0.07912306, ...,  0.02449062,
        -0.12757981,  0.05059064],
       [-0.06841816,  0.00492897, -0.15028928, ..., -0.04976881,
        -0.10502116, -0.06508294],
       ..., 
       [-0.12263435, -0.14896888, -0.14164306, ...,  0.09722459,
         0.09214186, -0.04767206],
       [-0.00329463, -0.03244751, -0.00636163, ..., -0.03522604,
         0.2135247 ,  0.18546703],
       [-0.04546939, -0.02252209, -0.06355116, ..., -0.1523219 ,
        -0.11625706,  0.04690878]], dtype=float32)]
